In [1]:
import numpy as np

### Aufgabe 1
(2+2+2 = 6 Punkte)

Approximation von Eigenwerten mit der QR-Zerlegung.
Implementieren Sie die folgenden Aufgaben in Python

a) Führen Sie für die Matrix 

```
    1   2   4
A = 2   3   8
    -1  -3  -1
```

eine QR-Zerlegung (numpy.linalg.qr) durch. Berechnen Sie die Probe (numpy.matmul).

In [2]:
A = [[1, 2, 4],
     [2, 3, 8],
     [-1, -3, -1]]

q, r = np.linalg.qr(A)

print("A\n", np.array(A))
print("\nQ\n", q)
print("\nR\n", r)
print("\nQ @ R\n", np.matmul(q, r)) # q@r


A
 [[ 1  2  4]
 [ 2  3  8]
 [-1 -3 -1]]

Q
 [[-0.40824829  0.12309149  0.90453403]
 [-0.81649658 -0.49236596 -0.30151134]
 [ 0.40824829 -0.86164044  0.30151134]]

R
 [[-2.44948974 -4.4907312  -8.5732141 ]
 [ 0.          1.3540064  -2.58492131]
 [ 0.          0.          0.90453403]]

Q @ R
 [[ 1.  2.  4.]
 [ 2.  3.  8.]
 [-1. -3. -1.]]


b) In der Vorlesung wurde ein Algorithmus besprochen, mit dem (auf Basis einer
QR-Zerlegung) aus einer gegebenen Matrix A eine ¨ahnliche, diagonaldominante Matrix A′
berechnet werden kann. Ziel der Berechnung von A′ ist die Approximation der Eigenwerte von
A durch die Diagonalelemente von A′. Implementieren Sie eine Funktion, die den in der
Vorlesung besprochenen Algorithmus durchf ¨uhrt und die Matrix A′ und die
Transformationsmatrix U zur ¨uckgibt

In [3]:
def diagonalize_qr(mat, it=10, verbose=False):
    a = mat
    u = np.eye(mat.shape[0])
    for n in range(it):
        if verbose:
            print(f"==========\n#{n=}")
            print(f'a_{n}=\n{a}')
        
        q, r = np.linalg.qr(a)
        if verbose:
            print(f'q_{n}=\n{q}\nr_{n}=\n{r}')
        
        a = r @ q # a_n+1
        if verbose:
            print(f'a_{n+1}=\n{a}\n')
        
        u = u @ q
    return a, u


c) Wenden Sie die Funktion auf die Matrix B = [...] an. Berechnen Sie die Probe B = U B′U T und vergleichen Sie Ihre Approximation der
Eigenwerte von B mit den Ergebnissen auf den Folien zur Vorlesung. Vergleichen Sie Ihr
Ergebnis zus ¨atzlich mit den Ergebnissen der Python-Funktion numpy.linalg.eigvals zur
Berechnung von Eigenwerten.

In [4]:

B = np.array([[2, -1], [-1, 4]])

print(B)


b_diag, u = diagonalize_qr(B, 100)

print("\nDiagonalized B (eigenvalues)\n", b_diag)
print("\nDiagonals (eigenvalues)\n", np.diag(b_diag))
print("\nEigenvectors (columns of U)\n", u)

print("==============================")

print("\nCheck: U @ B' @ U^T\n", u @ b_diag @ u.T)

print("==============================")

np_eigvals, np_eigvecs = np.linalg.eig(B)
print("\nEigenvalues from numpy.linalg.eig\n", np_eigvals)
print("\nEigenvectors from numpy.linalg.eig\n", np_eigvecs)



[[ 2 -1]
 [-1  4]]

Diagonalized B (eigenvalues)
 [[ 4.41421356e+00  8.00166881e-16]
 [-2.36296085e-44  1.58578644e+00]]

Diagonals (eigenvalues)
 [4.41421356 1.58578644]

Eigenvectors (columns of U)
 [[ 0.38268343  0.92387953]
 [-0.92387953  0.38268343]]

Check: U @ B' @ U^T
 [[ 2. -1.]
 [-1.  4.]]

Eigenvalues from numpy.linalg.eig
 [1.58578644 4.41421356]

Eigenvectors from numpy.linalg.eig
 [[-0.92387953  0.38268343]
 [-0.38268343 -0.92387953]]


In [5]:
def solveUpper(R, b):
    """ Solve Rx = b via back-substitution
    R is upper-triangular.
    """
    n = R.shape[0]
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        var1 = np.dot(R[i], x)
        var2 = np.dot(R[i, i:], x[i:])
        var3 = sum(a*b for a, b in zip(R[i, i:], x[i:]))
        assert var1 == var2
        assert var2 == var3
        
        x[i] = (b[i] - var1) / R[i, i]
    return x

def test_solveUpper(a, b):
    expected = np.linalg.solve(a, b)
    x = solveUpper(a, b)
    assert np.isclose(x, expected).all(), f"x={x}, expected {expected}"


a1 =np.array([
    [4, 0, 0],
    [0, 3, 0],
    [0, 0, 2]
])
b1 = np.array([8, 9, 10])
test_solveUpper(a1, b1)

a2 = np.array([
    [1, 1, 1],
    [0, 1, 1],
    [0, 0, 1]
])
b2 = np.array([6, 3, 1])
test_solveUpper(a2, b2)

a3 = np.array([
    [1, 2, 3],
    [0, 4, 5],
    [0, 0, 6]
])
b3 = np.array([14, 23, 18])
test_solveUpper(a3, b3)

In [6]:
def myQRSolver(A, b):
    q, r = np.linalg.qr(A)
    rhs = q.T @ b
    return solveUpper(r, rhs)

# c)

def test_c():
    A = np.array([
        [1, 2],
        [3, 1],
    ])
    b = np.array([1, 2])
    
    x = myQRSolver(A, b) # my solution
    print("myQRSolver(A, b) =", x)
    
    expectation = np.linalg.solve(A, b)
    print("Verify with numpy.linalg.solve(A, b) =", expectation)
    
    assert type(x) == type(expectation), f"x type={type(x)}, expected type {type(expectation)}"
    assert np.isclose(x, expectation).all(), f"x={x}, expected {expectation}"
    
test_c()

myQRSolver(A, b) = [0.6 0.2]
Verify with numpy.linalg.solve(A, b) = [0.6 0.2]
